In [ ]:
import numpy as np
import pandas as pd

var_CF = "CF"
var_BS = "BS"
var_IS = "IS"
var_FS = var_IS

folder_mai = "edgarfiles/"
folder_pas = "common files"
folder_fel = "this folder has the quarterly and annual files"
folder_mai = "common files are here"
folder_ann = "this folder has three subfodler CF, BS, IS for the three financial statements"
folder_qtr = "this folder has three subfodler CF, BS, IS for the three quarterly financial statements"

In [ ]:
# Get all Quarterly Entries
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str,"ffy":str,"line":str}
df_QIS = pd.read_csv((folder_fel+"10Q_INCOME.csv.zip"),dtype=df_col_types)
df_QIS.drop(["Unnamed: 0"],axis=1,inplace=True)
print(len(df_QIS))

df_QBS = pd.read_csv((folder_fel+"10Q_BALANC.csv.zip"),dtype=df_col_types)
df_QBS.drop(["Unnamed: 0"],axis=1,inplace=True)
print(len(df_QBS))

df_QCF = pd.read_csv((folder_fel+"10Q_CASHFL.csv.zip"),dtype=df_col_types)
df_QCF.drop(["Unnamed: 0"],axis=1,inplace=True)
print(len(df_QCF))

df_currency = pd.read_excel((folder_mai+"Currency Data.xlsx"),sheet_name="subset")

# GET ALL TAGS
df_all_tags = pd.read_excel((folder_mai+"Tags and Stats.xlsx"),sheet_name="Tag List")
df_all_tags.sort_values(by=["stmt","metric","tag_order"],ascending=True,inplace=True)
print(len(df_all_tags))

# GET THE INDUSTRY AND THE SECTOR
df_cik_sic_align = pd.read_csv((folder_mai+"cik_sic_align.csv"),dtype=df_col_types)
df_sic_data = df_cik_sic_align[["sic","Office","Industry Title"]] 
df_sic_data.drop_duplicates(subset="sic",inplace=True)


# Data of a Single Company

In [ ]:
var_co_name = "APPLE IN"
df_10Q_BS_temp = df_QBS.copy()
df_10Q_IS_temp = df_QIS.copy()
df_10Q_CF_temp = df_QCF.copy()

df_co_data = df_10Q_BS_temp[df_10Q_BS_temp["name"].str.contains(var_co_name)]
df_co_data.drop_duplicates(subset=["name"],inplace=True)
df_co_data.head(2)


In [ ]:
# Get the data of the Foreign company 
var_co_name = "AAPL"
var_cik = "320193"
df_10Q_BS_temp = df_QBS.copy()
df_10Q_IS_temp = df_QIS.copy()
df_10Q_CF_temp = df_QCF.copy()
df_co_bs = df_10Q_BS_temp[df_10Q_BS_temp["cik"].str.contains(var_cik)]
df_co_is = df_10Q_IS_temp[df_10Q_IS_temp["cik"].str.contains(var_cik)]
df_co_cf = df_10Q_CF_temp[df_10Q_CF_temp["cik"].str.contains(var_cik)]

df_co = df_co_bs
df_co = df_co.append(df_co_is,ignore_index=True)
df_co = df_co.append(df_co_cf,ignore_index=True)

df_co.to_csv((folder_cos+var_co_name+" "+var_cik+" QUART.csv"))

In [ ]:
# Get the data of the Foreign company 
var_co_name = "AAPL"
var_cik = "320193"
df_FS_temp = df_QCF.copy()
df_co_fs = df_FS_temp[df_FS_temp["cik"].str.contains(var_cik)]

df_co_fs.to_csv((folder_fel+var_co_name+" "+var_cik+" QUART.csv"))

# Financial Statement Entries

In [ ]:
var_FS = var_IS
get_financial_statement_elements(var_FS,df_QIS)

var_FS = var_BS
get_financial_statement_elements(var_FS,df_QBS)

var_FS = var_CF
get_financial_statement_elements(var_FS,df_QCF)

In [ ]:
def get_financial_statement_elements(var_FS,df_QUART):

    # GET METRICs 
    ls_Qs = ["Q1","Q2","Q3"]
    ls_met_tags = pd.unique(df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)])

    counter_met = 0

    while counter_met < len(ls_met_tags):

    # GET RELEVANT TAGS    
        df_rel_tags = df_all_tags[(df_all_tags["metric"] == ls_met_tags[counter_met])&((df_all_tags["stmt"] == var_FS))]

        df_rel_tags.drop_duplicates(inplace=True)
        df_rel_tags = pd.DataFrame(df_rel_tags)
        print(("Tags in Metric "+ls_met_tags[counter_met]+" are "+np.str(len(df_rel_tags))))
        
    # PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
        df_QUARTR_temp = df_QUART.copy()
        
        if var_FS == var_BS:
            df_QUARTR_rel = df_QUARTR_temp[(df_QUARTR_temp["stmt"] == var_FS)&(df_QUARTR_temp["fp"].isin(ls_Qs))
                                           &(df_QUARTR_temp["tag"].isin(df_rel_tags["tag"]))]
        else:
            df_QUARTR_rel = df_QUARTR_temp[(df_QUARTR_temp["stmt"] == var_FS)&(df_QUARTR_temp["fp"].isin(ls_Qs))
                                       &(df_QUARTR_temp["tag"].isin(df_rel_tags["tag"]))]

        print(("Entries with "+ls_met_tags[counter_met]+" relevant tags are "+np.str(len(df_QUARTR_rel))))

        if ls_met_tags[counter_met] == "CASH":      
            li_cash_tags = ["MARKETABLESECURITIESCURRENT","AVAILABLEFORSALESECURITIESCURRENT","SHORTTERMINVESTMENTS"]
            df_rel_tags = df_rel_tags[~(df_rel_tags["tag"].isin(li_cash_tags))]
            ls_rel_tags = pd.unique(df_rel_tags["tag"])
        else:
            ls_rel_tags = pd.unique(df_rel_tags["tag"])
        
        df_QUARTR_temp = df_QUARTR_rel.copy()
        df_rel_cos = df_QUARTR_temp[["cfp"]]
        df_rel_cos.drop_duplicates(subset=["cfp"],inplace=True)
        df_QUARTR_temp = pd.DataFrame()
        df_rel_cos_temp = df_rel_cos.copy()
        print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))

        # Create a copy of the relevant entries
        df_QUARTR_rcop = df_QUARTR_rel.copy()

        df_entries_temp = pd.DataFrame()
        df_cosfy_tagfound = pd.DataFrame()
        df_cosfy_alltagfound = pd.DataFrame()
        df_cf_list = pd.DataFrame()
        counter_tag = 0

        if ls_met_tags[counter_met] == "NETCASH":
# NETCASH in CF are unsual entries, special logic for them
            ls_opr = ["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES","NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"]
            df_cosfy_tagfound = get_netcash(ls_opr,df_QUARTR_rel)
            df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)

            ls_inv = ["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES","NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"]
            df_cosfy_tagfound = get_netcash(ls_inv,df_QUARTR_rel)
            df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)

            ls_fin = ["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES","NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"]
            df_cosfy_tagfound = get_netcash(ls_fin,df_QUARTR_rel)
            df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
            
        else:
            while counter_tag < len(ls_rel_tags):

                if ls_met_tags[counter_met] == "CASH":

                    df_entries_temp = df_QUARTR_rcop[df_QUARTR_rcop["cfp"].isin(df_rel_cos_temp["cfp"])]
                    ls_cash_tags = np.append(ls_rel_tags[counter_tag],li_cash_tags)
                    df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"].isin(ls_cash_tags))]

                else:

                    df_entries_temp = df_QUARTR_rcop[df_QUARTR_rcop["cfp"].isin(df_rel_cos_temp["cfp"])]
                    df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"] == ls_rel_tags[counter_tag])]

                df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)

                df_cf_list = df_cosfy_tagfound[["cfp"]]
                df_cf_list.drop_duplicates()  
                df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfp"].isin(df_cf_list["cfp"]))]
                print((ls_rel_tags[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))

                counter_tag = counter_tag + 1

# Q4 will be computed from the Annual File. Get the Annual File for the same metric 
        df_ANN = pd.read_csv((folder_ann+var_FS+"/"+ls_met_tags[counter_met]+".csv"),dtype=df_col_types)
        df_ANN.drop(["Unnamed: 0"],axis=1,inplace=True)

        if var_FS == var_IS:
# In the Income statement, Q4 doesn't have all financial elements, aggregate values of Q1 to Q3 and subtract from Year
# Get the Q3 entries (this helps ensure that the Q4 entries have the same columns as Q1 to Q3
            df_Q3 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q3"]
            df_Q13 = df_cosfy_alltagfound[["cfy","value"]]
            df_Q13x = df_Q13.groupby(["cfy"]).sum().reset_index()
            df_Q4x = df_ANN[["cfy","fdate","qtrs","value"]]
            df_Q4y = pd.merge(df_Q3,df_Q13x,on="cfy",how="left")
            df_Q4y.rename(columns={"value_y":"F3QSum"},inplace=True)
            df_Q4 = pd.merge(df_Q4y,df_Q4x,on="cfy",how="left")            
            df_Q4["fdate"] = df_Q4["fdate_y"]
            df_Q4["qtrs"] = df_Q4["qtrs_y"]
            df_Q4["value"] = df_Q4["value"] - df_Q4["F3QSum"]
            df_Q4["cfp"] = df_Q4["cfy"].astype(str) + "-Q4"
            df_Q4["fyp"] = df_Q4["ffy"].astype(str) + "-Q4"
            df_Q4["fp"] = "Q4"
            df_Q4.drop(["fdate_y","fdate_x","qtrs_y","qtrs_x","value_x","F3QSum"],axis=1,inplace=True)            
            df_QTR = df_cosfy_alltagfound.append(df_Q4,ignore_index=False)
        elif var_FS == var_BS:
# In the Balance Sheet, copy annual BS into Q4 and append
            df_Q4 = df_ANN
            df_Q4["fp"] = "Q4"
            df_Q4["fyp"] = df_Q4["ffy"].astype(str) + "-Q4"
            df_Q4["cfp"] = df_Q4["cfy"].astype(str) + "-Q4"
            df_Q4["fp"] = "Q4"
            df_Q4["fy"] = df_Q4["ffy"]
            df_Q4["calyr"] = df_Q4["ffy"]
            df_QTR = df_cosfy_alltagfound.append(df_Q4,ignore_index=False)
            df_QTR.drop(["Office","Industry Title","inUSD","value Bn USD","Currency"],axis=1,inplace=True)       
        else:
            df_QTR = pd.DataFrame()
# The Quarterly entries in Cash Flows are cumulative (Q2 = Q1+Q2, Q3 = Q1+Q2+Q3 etc.), rolling calculation to be done
            if ls_met_tags[counter_met] == "NETCASH":
      # Get the Q3 Entries
                df_Q3 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q3"]
    # Merge the Annual Entries
                df_Q4x = df_ANN[["cfy","tag","fdate","qtrs","value"]]
    # Compute Q4
                df_Q4 = pd.merge(df_Q3,df_Q4x,on=["cfy","tag"],how="left")
                df_Q4["fdate"] = df_Q4["fdate_y"]
                df_Q4["qtrs"] = df_Q4["qtrs_y"]
                df_Q4["value"] = df_Q4["value_y"] - df_Q4["value_x"]
                df_Q4["cfp"] = df_Q4["cfy"].astype(str) + "-Q4"
                df_Q4["fyp"] = df_Q4["ffy"].astype(str) + "-Q4"
                df_Q4["fp"] = "Q4"
                df_Q4.drop(["fdate_y","fdate_x","qtrs_y","qtrs_x","value_x","value_y"],axis=1,inplace=True)
                df_QTR = df_QTR.append(df_Q4,ignore_index=False)
    # Compute Q3
                df_Q2 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q2"]
                df_Q2x = df_Q2[["cfy","tag","value"]]
                df_Q3x = pd.merge(df_Q3,df_Q2x,on=["cfy","tag"],how="left")
                df_Q3x["value"] = df_Q3x["value_x"] - df_Q3x["value_y"]
                df_Q3x.drop(["value_x","value_y"],axis=1,inplace=True)
                df_QTR = df_QTR.append(df_Q3x,ignore_index=False)
    # Compute Q2
                df_Q1 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q1"]
                df_Q1x = df_Q1[["cfy","tag","value"]]
                df_Q2x = pd.merge(df_Q2,df_Q1x,on=["cfy","tag"],how="left")
                df_Q2x["value"] = df_Q2x["value_x"] - df_Q2x["value_y"]
                df_Q2x.drop(["value_x","value_y"],axis=1,inplace=True)
                df_QTR = df_QTR.append(df_Q2x,ignore_index=False)
    # Append Q1
                df_QTR = df_QTR.append(df_Q1,ignore_index=False)          
        
            else:
    
    # Get the Q3 Entries
                df_Q3 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q3"]
    # Merge the Annual Entries
                df_Q4x = df_ANN[["cfy","fdate","qtrs","value"]]
    # Compute Q4
                df_Q4 = pd.merge(df_Q3,df_Q4x,on="cfy",how="left")
                df_Q4["fdate"] = df_Q4["fdate_y"]
                df_Q4["qtrs"] = df_Q4["qtrs_y"]
                df_Q4["value"] = df_Q4["value_y"] - df_Q4["value_x"]
                df_Q4["cfp"] = df_Q4["cfy"].astype(str) + "-Q4"
                df_Q4["fyp"] = df_Q4["ffy"].astype(str) + "-Q4"
                df_Q4["fp"] = "Q4"
                df_Q4.drop(["fdate_y","fdate_x","qtrs_y","qtrs_x","value_x","value_y"],axis=1,inplace=True)
                df_QTR = df_QTR.append(df_Q4,ignore_index=False)
    # Compute Q3
                df_Q2 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q2"]
                df_Q2x = df_Q2[["cfy","value"]]
                df_Q3x = pd.merge(df_Q3,df_Q2x,on="cfy",how="left")
                df_Q3x["value"] = df_Q3x["value_x"] - df_Q3x["value_y"]
                df_Q3x.drop(["value_x","value_y"],axis=1,inplace=True)
                df_QTR = df_QTR.append(df_Q3x,ignore_index=False)
    # Compute Q2
                df_Q1 = df_cosfy_alltagfound[df_cosfy_alltagfound["fp"] == "Q1"]
                df_Q1x = df_Q1[["cfy","value"]]
                df_Q2x = pd.merge(df_Q2,df_Q1x,on="cfy",how="left")
                df_Q2x["value"] = df_Q2x["value_x"] - df_Q2x["value_y"]
                df_Q2x.drop(["value_x","value_y"],axis=1,inplace=True)
                df_QTR = df_QTR.append(df_Q2x,ignore_index=False)
    # Append Q1
                df_QTR = df_QTR.append(df_Q1,ignore_index=False)
    
        df_QTR = pd.merge(df_QTR,df_currency,left_on="uom",right_on="Currency",how="left")
        df_QTR["value Bn USD"] = df_QTR["value"] / df_QTR["inUSD"] / 1000000000

        # MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
        df_QTR = pd.merge(df_QTR,df_sic_data,on="sic",how="left")

        print("Entries Found "+np.str(len(df_QTR)))
        df_QTR.to_csv((folder_qtr+var_FS+"/"+ls_met_tags[counter_met]+".csv"))
        
        counter_met = counter_met + 1

    return

In [ ]:
def get_netcash(ls_netcas,df_QTR_NC):
    
    df_netcash_entries = df_QTR_NC[(df_QTR_NC["tag"].isin(ls_netcas))]
    df_netcash_entries.sort_values(by=["cfp","value"],inplace=True)
    df_netcash_entries.drop_duplicates(subset="cfp",inplace=True)

    return(df_netcash_entries)